# NQL Skeleton Code
Imports, hyperparameters and environment setup

In [1]:
import gym
import tensorflow as tf
import numpy as np
import random

GAMMA = 0.9  # discount factor for target Q
INITIAL_EPSILON = 0.6  # starting value of epsilon
FINAL_EPSILON = 0.05  # final value of epsilon
EPSILON_DECAY_STEPS = 1000
TEST_FREQUENCY = 100  # Num episodes to run before visualizing test accuracy
HIDDEN_NODES = 20
ENV_NAME = 'CartPole-v0'
EPISODE = 200000  # Episode limitation
STEP = 200  # Step limitation in an episode
TEST = 10  # The number of tests to run every TEST_FREQUENCY episodes
env = gym.make(ENV_NAME)
epsilon = INITIAL_EPSILON
STATE_DIM = env.observation_space.shape[0]
ACTION_DIM = env.action_space.n

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


Define our inputs

In [2]:
# TODO: Add shape argument to placeholder. Hint: for state_in it will be [None, STATE_DIM]
state_in = tf.placeholder("float", )
action_in = tf.placeholder("float", ) 
target_in = tf.placeholder("float", )

Define our network

In [4]:
# TODO: Add the units args and activation functions for the h2 and q_values layers
h1 = tf.layers.dense(state_in, HIDDEN_NODES, activation=tf.nn.relu)
h2 = tf.layers.dense(h1, ...

q_values = tf.layers.dense(h2, ...

q_action = tf.reduce_sum(tf.multiply(q_values, action_in), reduction_indices=1)

Define our loss function

In [5]:
# TODO: Define the loss in terms of target_in and q_action tensors
loss = ...
optimizer = tf.train.AdamOptimizer(0.0003).minimize(loss)

Define our exploration policy

In [6]:
# NOTE: assumes all our previously defined vars are accessible
# TODO: using the generate Q-values, implement an e-greedy exploration policy from earlier in the workshop.
# the return value must be a 1-hot representation of the action
def explore(state, epsilon):
    Q_estimates = q_values.eval(feed_dict={
        state_in: [state]
    })
    ...
    return one_hot_action

Initialize Network

In [7]:
# Start session - Tensorflow housekeeping
session = tf.InteractiveSession()
session.run(tf.global_variables_initializer())

Now our main loop

In [8]:
for episode in range(EPISODE):
    # initialize task
    state = env.reset()

    # Update epsilon once per episode - linear schedule
    epsilon -= epsilon / EPSILON_DECAY_STEPS

    # Move through env according to e-greedy policy
    for step in range(STEP):
        action = explore(state, epsilon)

        #TODO: Take an env step. Hint: env.step() takes an integer representing an action
        next_state, reward, done, _ = ...

        #TODO: get the q_values of the next state. Hint: look how we get them in the explore funciton
        nextstate_q_values = ...
        
        if done:
            target = reward
        else:
            #TODO: Calculate the target value when episode is not done
            target = ...

        # Do our train step
        session.run([optimizer], feed_dict={
            target_in: [target],
            action_in: [action],
            state_in: [state]
        })
        
        #TODO: Update the state and terminate episode if done
        ...

    # Test and view sample runs - can disable render to save time
    if (episode % TEST_FREQUENCY == 0 and episode != 0):
        total_reward = 0
        for i in range(TEST):
            state = env.reset()
            for j in range(STEP):
                env.render()
                action = np.argmax(q_values.eval(feed_dict={
                    state_in: [state]
                    }))
                state, reward, done, _ = env.step(action)
                total_reward += reward
                if done:
                    break
        ave_reward = total_reward / TEST
        print('episode:', episode, 'epsilon:', epsilon, 'Average Reward:', ave_reward)
env.close()

episode: 100 epsilon: 0.5423324129799574 Evaluation Average Reward: 9.3
episode: 200 epsilon: 0.4906981083894946 Evaluation Average Reward: 9.6
episode: 300 epsilon: 0.4439797950743664 Evaluation Average Reward: 45.0


KeyboardInterrupt: 